In [1]:
#upload do arquivo .xlxs

from google.colab import files
uploaded = files.upload()

Saving Bases Clientes e Vendas - Case Dados.xlsx to Bases Clientes e Vendas - Case Dados.xlsx


In [2]:
#biblioteca pandas + as duas sheets existentes da planilha

import pandas as pd
arquivo_excel = "Bases Clientes e Vendas - Case Dados.xlsx"

cadastro = pd.read_excel(arquivo_excel, sheet_name='Cadastro')
vendas = pd.read_excel(arquivo_excel, sheet_name='Vendas')

print("Cadastro")
display(cadastro.head())

print("Vendas")
display(vendas.head())

Cadastro


,COD_CLIENTE,DATA_CADASTRO,DATA_NASCIMENTO,CELULAR_VALIDO,E_MAIL_VALIDO,OPT_OUT
0,39686368,45468,19710410,Não,Sim,Não
1,39116651,45294,19970108,Não,Sim,Não
2,1061626,45394,19811230,Não,Sim,Sim
3,39819563,45511,19891103,Não,Sim,Não
4,39347635,45362,19660508,Não,Sim,Não


Vendas


,COD_CLIENTE,CANAL_VENDA,COD_CUPOM,DATA_VENDA,IMPOSTO_VENDA,QUANTIDADE_VENDA,VALOR_VENDA,AVALIACAO_ATENDIMENTO
0,39149880,3000,04 02260000781381,20250205,14.85,1,67.49,6
1,40322711,1000,04 0010000129633A,20250113,19.95,1,220.00,10
2,39313219,3000,04 00690001988051,20250331,20.33,1,79.50,1
3,39154261,4300,04 00310002592671,20250402,17.70,1,94.69,7
4,39572507,1000,04 0150000044376104,20250420,1.02,1,10.99,5


In [3]:
#criar um código para informar a faixa de idade

from datetime import datetime
import numpy as np

cadastro = pd.read_excel(arquivo_excel, sheet_name='Cadastro', dtype={'DATA_NASCIMENTO': str})

cadastro['DATA_NASCIMENTO'] = pd.to_datetime(cadastro['DATA_NASCIMENTO'], format='%Y%m%d', errors='coerce')

hoje = pd.to_datetime('today')
cadastro['Idade'] = (hoje - cadastro['DATA_NASCIMENTO']).dt.days // 365

bins = [0, 20, 30, 40, 50, np.inf]
labels = ['<=20', '21 a 30', '31 a 40', '41 a 50', '>=51']
cadastro['Faixa_Idade'] = pd.cut(cadastro['Idade'], bins=bins, labels=labels)

cadastro[['COD_CLIENTE', 'DATA_NASCIMENTO', 'Idade', 'Faixa_Idade']].head()

,COD_CLIENTE,DATA_NASCIMENTO,Idade,Faixa_Idade
0,39686368,1971-04-10,54,>=51
1,39116651,1997-01-08,28,21 a 30
2,1061626,1981-12-30,43,41 a 50
3,39819563,1989-11-03,35,31 a 40
4,39347635,1966-05-08,59,>=51


In [4]:
#trazer a média de dias até a primeira compra, converter a data de cadastro

cadastro['COD_CLIENTE'] = cadastro['COD_CLIENTE'].astype(str)
vendas['COD_CLIENTE'] = vendas['COD_CLIENTE'].astype(str)

cadastro['DATA_CADASTRO'] = pd.to_datetime(cadastro['DATA_CADASTRO'], origin='1899-12-30', unit='D')
vendas['DATA_VENDA'] = pd.to_datetime(vendas['DATA_VENDA'], format='%Y%m%d', errors='coerce')


primeira_compra = vendas.groupby('COD_CLIENTE')['DATA_VENDA'].min().reset_index()
primeira_compra.rename(columns={'DATA_VENDA': 'DATA_PRIMEIRA_COMPRA'}, inplace=True)
clientes_compras = pd.merge(cadastro, primeira_compra, on='COD_CLIENTE', how='inner')
clientes_compras['DIAS_ATE_1A_COMPRA'] = (clientes_compras['DATA_PRIMEIRA_COMPRA'] - clientes_compras['DATA_CADASTRO']).dt.days
media_dias = clientes_compras['DIAS_ATE_1A_COMPRA'].mean()
print(f"Média de dias até a primeira compra: {media_dias:.2f} dias")

Média de dias até a primeira compra: 144.50 dias


In [5]:
#trazer a média de clientes por mês + o seu respectivo canal de vendas + média de clientes

vendas['ANO_MES'] = vendas['DATA_VENDA'].dt.to_period('M')
clientes_por_mes = vendas.groupby(['ANO_MES', 'CANAL_VENDA'])['COD_CLIENTE'].nunique().reset_index(name='CLIENTES_UNICOS')
media_clientes_mes = clientes_por_mes.groupby('CANAL_VENDA')['CLIENTES_UNICOS'].mean().reset_index(name='MEDIA_CLIENTES_MES')
print(media_clientes_mes)

    CANAL_VENDA  MEDIA_CLIENTES_MES
0          1000          364.250000
1          2000           26.750000
2          2001            5.750000
3          2002            2.333333
4          2003            1.750000
5          3000          304.000000
6          4100           76.750000
7          4201            1.000000
8          4300          132.750000
9          4400            1.000000
10         4401           17.750000
11         4402            1.000000
12         4500            5.750000
13         4600           39.000000
14         7300            3.500000


In [6]:
#Trazer a média de tickets (compras) de clientes por mês

tickets_por_mes = vendas.groupby(['ANO_MES', 'CANAL_VENDA']).size().reset_index(name='QTDE_TICKETS')
media_tickets_mes = tickets_por_mes.groupby('CANAL_VENDA')['QTDE_TICKETS'].mean().reset_index(name='MEDIA_TICKETS_MES')
print(media_tickets_mes)

    CANAL_VENDA  MEDIA_TICKETS_MES
0          1000            1521.50
1          2000              61.75
2          2001              10.00
3          2002               6.00
4          2003               3.25
5          3000             626.75
6          4100             145.00
7          4201               1.00
8          4300             294.00
9          4400               1.00
10         4401              29.00
11         4402               1.00
12         4500              16.25
13         4600              79.00
14         7300              10.00


In [7]:
#ticket médio por mês

resumo_geral = vendas.groupby('ANO_MES').agg({ 'VALOR_VENDA': 'sum','COD_CLIENTE': 'count'}).reset_index()
resumo_geral['TICKET_MEDIO'] = resumo_geral['VALOR_VENDA'] / resumo_geral['COD_CLIENTE']
print(resumo_geral[['ANO_MES', 'TICKET_MEDIO']])

   ANO_MES  TICKET_MEDIO
0  2025-01     63.039565
1  2025-02     60.227823
2  2025-03     65.142824
3  2025-04     64.448472


In [8]:
#gasto médio por mês

faturamento_mensal = vendas.groupby(['CANAL_VENDA', 'ANO_MES'])['VALOR_VENDA'].sum().reset_index()

gasto_medio_mensal = faturamento_mensal.groupby('CANAL_VENDA')['VALOR_VENDA'].mean().reset_index()
gasto_medio_mensal.rename(columns={'VALOR_VENDA': 'GASTO_MEDIO_MES'}, inplace=True)
print(gasto_medio_mensal)

    CANAL_VENDA  GASTO_MEDIO_MES
0          1000     69332.610000
1          2000      6165.705000
2          2001       974.170000
3          2002       203.826667
4          2003       129.697500
5          3000     54565.845000
6          4100     13544.532500
7          4201       119.455000
8          4300     22679.320000
9          4400       109.840000
10         4401      3576.760000
11         4402        30.490000
12         4500       706.722500
13         4600      4749.025000
14         7300       705.790000


In [9]:
#Qual a frequência média por mês

compras_por_mes = vendas.groupby(['CANAL_VENDA', 'ANO_MES']).size().reset_index(name='NUM_COMPRAS')

clientes_por_mes = vendas.groupby(['CANAL_VENDA', 'ANO_MES'])['COD_CLIENTE'].nunique().reset_index(name='CLIENTES_UNICOS')

frequencia_df = pd.merge(compras_por_mes, clientes_por_mes, on=['CANAL_VENDA', 'ANO_MES'])

frequencia_df['FREQUENCIA_MES'] = frequencia_df['NUM_COMPRAS'] / frequencia_df['CLIENTES_UNICOS']

frequencia_media = frequencia_df.groupby('CANAL_VENDA')['FREQUENCIA_MES'].mean().reset_index()
frequencia_media.rename(columns={'FREQUENCIA_MES': 'FREQUENCIA_MEDIA_MES'}, inplace=True)
print(frequencia_media)

    CANAL_VENDA  FREQUENCIA_MEDIA_MES
0          1000              4.183767
1          2000              2.290488
2          2001              2.105769
3          2002              2.388889
4          2003              2.125000
5          3000              2.059827
6          4100              1.894490
7          4201              1.000000
8          4300              2.210054
9          4400              1.000000
10         4401              1.669625
11         4402              1.000000
12         4500              2.927083
13         4600              2.023896
14         7300              3.312500


In [10]:
# O principal canal de vendas

faturamento_por_canal = vendas.groupby('CANAL_VENDA')['VALOR_VENDA'].sum().reset_index()

faturamento_por_canal = faturamento_por_canal.sort_values(by='VALOR_VENDA', ascending=False)

principal_canal = faturamento_por_canal.iloc[0]

print(f"Principal canal de venda é: {principal_canal['CANAL_VENDA']} com faturamento total de R${principal_canal['VALOR_VENDA']:.2f}")

Principal canal de venda é: 1000.0 com faturamento total de R$277330.44


In [11]:
# Qual o faturamento total

faturamento_por_canal = vendas.groupby('CANAL_VENDA')['VALOR_VENDA'].sum().reset_index()
faturamento_por_canal.rename(columns={'VALOR_VENDA': 'FATURAMENTO_TOTAL'}, inplace=True)

print(" Faturamento total por canal de venda é:")
print(faturamento_por_canal)

faturamento_geral = faturamento_por_canal['FATURAMENTO_TOTAL'].sum()
print(f"\n Faturamento geral (todos os canais é): R$ {faturamento_geral:,.2f}")

 Faturamento total por canal de venda é:
    CANAL_VENDA  FATURAMENTO_TOTAL
0          1000          277330.44
1          2000           24662.82
2          2001            3896.68
3          2002             611.48
4          2003             518.79
5          3000          218263.38
6          4100           54178.13
7          4201             238.91
8          4300           90717.28
9          4400             219.68
10         4401           14307.04
11         4402              60.98
12         4500            2826.89
13         4600           18996.10
14         7300            2823.16

 Faturamento geral (todos os canais é): R$ 709,651.76


In [12]:
#Total de imposto geral

imposto_por_canal = vendas.groupby('CANAL_VENDA')['IMPOSTO_VENDA'].sum().reset_index()
imposto_por_canal.rename(columns={'IMPOSTO_VENDA': 'TOTAL_IMPOSTO'}, inplace=True)

print(" Total de imposto gerado por canal de venda é:")
print(imposto_por_canal)

imposto_geral = imposto_por_canal['TOTAL_IMPOSTO'].sum()
print(f"\n Total geral de imposto gerado: R$ {imposto_geral:,.2f}")

 Total de imposto gerado por canal de venda é:
    CANAL_VENDA  TOTAL_IMPOSTO
0          1000       51634.47
1          2000        5353.47
2          2001         929.42
3          2002         236.44
4          2003         133.84
5          3000       32305.91
6          4100        9774.88
7          4201          39.69
8          4300       19900.00
9          4400          92.22
10         4401        2204.66
11         4402          16.40
12         4500         471.07
13         4600        3121.64
14         7300         461.91

 Total geral de imposto gerado: R$ 126,676.02


In [13]:
# Nota de NPS

vendas_nps = vendas[vendas['AVALIACAO_ATENDIMENTO'].between(0, 10)]

def classificar_nps(nota):
    if nota >= 9:
        return 'Promotor'
    elif nota >= 7:
        return 'Neutro'
    else:
        return 'Detrator'

vendas_nps['NPS_TIPO'] = vendas_nps['AVALIACAO_ATENDIMENTO'].apply(classificar_nps)

nps_mensal = vendas_nps.groupby(['ANO_MES', 'NPS_TIPO']).size().unstack(fill_value=0).reset_index()

nps_mensal['TOTAL'] = nps_mensal[['Promotor', 'Detrator', 'Neutro']].sum(axis=1)

nps_mensal['NPS'] = ((nps_mensal['Promotor'] - nps_mensal['Detrator']) / nps_mensal['TOTAL']) * 100

print(nps_mensal[['ANO_MES', 'NPS']])

NPS_TIPO  ANO_MES        NPS
0         2025-01 -39.263158
1         2025-02 -42.262830
2         2025-03 -41.824324
3         2025-04 -40.134371


In [ ]:
import zipfile
import os

# Nome do arquivo zipado
zip_filename = "dados_case.zip"

# Cria o zip
with zipfile.ZipFile(zip_filename, "w") as zipf:
    arquivos = [
        "cadastro.csv",
        "vendas.csv",
        "resumo_geral.csv",
        "gasto_medio_mensal.csv",
        "frequencia_media.csv",
        "faturamento_por_canal.csv",
        "imposto_por_canal.csv",
        "nps_mensal.csv"
    ]
    for arquivo in arquivos:
        if os.path.exists(arquivo):
            zipf.write(arquivo)

            from google.colab import files
files.download("dados_case.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import zipfile
import os

os.makedirs("/content/csvs", exist_ok=True)

dataframes = {
    "cadastro": cadastro,
    "clientes_compras": clientes_compras,
    "clientes_por_mes": clientes_por_mes,
    "compras_por_mes": compras_por_mes,
    "faturamento_mensal": faturamento_mensal,
    "faturamento_por_canal": faturamento_por_canal,
    "frequencia_df": frequencia_df,
    "frequencia_media": frequencia_media,
    "gasto_medio_mensal": gasto_medio_mensal,
    "imposto_por_canal": imposto_por_canal,
    "media_clientes_mes": media_clientes_mes,
    "media_tickets_mes": media_tickets_mes,
    "nps_mensal": nps_mensal,
    "primeira_compra": primeira_compra,
    "resumo_geral": resumo_geral,
    "tickets_por_mes": tickets_por_mes,
    "vendas": vendas,
    "vendas_nps": vendas_nps
}

for nome, df in dataframes.items():
    caminho = f"/content/csvs/{nome}.csv"
    df.to_csv(caminho, index=False)

    zip_path = "/content/dataframes_case.zip"
with zipfile.ZipFile(zip_path, "w") as zipf:
    for nome in dataframes:
        zipf.write(f"/content/csvs/{nome}.csv", arcname=f"{nome}.csv")

        from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>